<a href="https://colab.research.google.com/github/BOD-27/Find_Missing_People/blob/main/Find_Missing_People_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/BOD-27/GP.git
import os
os.chdir('GP')
!unzip /content/GP/FinalTest_dataset.zip
!unzip /content/GP/Final_dataset.zip
!pip install streamlit
!pip install mtcnn
!pip install keras-facenet

from google.colab import drive
drive.mount('/content/drive')


Cloning into 'GP'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 104 (delta 51), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (104/104), 59.11 MiB | 22.67 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Archive:  /content/GP/FinalTest_dataset.zip
   creating: FinalTest_dataset/Abdo/
  inflating: FinalTest_dataset/Abdo/Abdo.jpg  
  inflating: FinalTest_dataset/Abdo/Abdo1.png  
  inflating: FinalTest_dataset/Abdo/Abdo2.png  
  inflating: FinalTest_dataset/Abdo/Abdo3.png  
  inflating: FinalTest_dataset/Abdo/Abdo4.png  
  inflating: FinalTest_dataset/Abdo/Abdo5.png  
  inflating: FinalTest_dataset/Abdo/Abdo6.png  
   creating: FinalTest_dataset/Ahmed/
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (1).jpg  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (1).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (2).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (3).

In [ ]:
%%writefile app.py

import os
import sys
import shutil
import streamlit as st
from PIL import Image
import subprocess
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
import numpy as np

from Life_Span_Model import clone_repo_and_install_deps, download_models, import_libraries, set_options, load_data_visualizer, switch_model_gender, process_image_and_save_result
from Face_Recognition import import_libraries as fr_import_libraries, suppress_tf_warnings, create_embedding_save_directory, FaceLoading, find_closest_matches, check_for_match

def main():
    st.title("Image Processing and Face Recognition")

    # File upload
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        # Save the uploaded file to disk
        img_path = os.path.join("/content", uploaded_file.name)
        with open(img_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Display the uploaded image
        st.image(Image.open(uploaded_file), caption="Uploaded Image", use_column_width=True)

        # Gender selection
        selected_gender = st.selectbox("Select gender for the lifespan model:", ["male", "female"])

        # Button to run the models
        if st.button("Run Models"):
            st.write("Running lifespan model...")
            clone_repo_and_install_deps()
            download_models()
            import_libraries()
            opt = set_options()
            dataset, visualizer = load_data_visualizer(opt)
            model = switch_model_gender(selected_gender, opt)
            image_folder_path = process_image_and_save_result(img_path, dataset, model, visualizer)
            st.write(f"Image processed and saved at: {image_folder_path}")

            st.write("Running face recognition model...")
            fr_import_libraries()
            suppress_tf_warnings()
            embedding_save_directory = create_embedding_save_directory()

            face_loader = FaceLoading('/content/GP/Final_dataset')
            embedded_x, dataset_labels = face_loader.load_or_encode_and_save_embeddings(embedding_save_directory)

            face_loader2 = FaceLoading(image_folder_path)
            X1, y1 = face_loader2.LoadClasses()

            # Debugging information
            print(f"Loaded images: {len(X1)}")

            if len(X1) == 0:
                st.error("No images found in the processed image folder. Please check the uploaded image and try again.")
                return

            embedder = FaceNet()
            embedded_x1 = [face_loader2.GetEmbedding(embedder, img) for img in X1]

            top_matches = find_closest_matches(embedded_x1, embedded_x, dataset_labels, top_n=7)

            st.write("Top Matches:")
            for i, (label, distance) in enumerate(top_matches, 1):
                st.write(f"Similar Image {i}: Label - {label}, Similarity Score - {distance}")

            match_found, avg_similarity, matched_label = check_for_match(top_matches)
            if match_found:
                st.write(f"Match found with label: {matched_label}")
                st.write(f"Average cosine similarity: {avg_similarity}")
            else:
                st.write("No match found.")
                st.write("Adding embeddings...")
                for i, img in enumerate(X1):
                    embedding = face_loader2.GetEmbedding(embedder, img)
                    label = y1[i]
                    np.save(os.path.join(embedding_save_directory, f'{label}embeddings.npy'), embedding)

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com


34.82.103.41


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.82.103.41:8501

npx: installed 22 in 3.598s
your url is: https://dirty-papers-judge.loca.lt
2024-06-02 18:14:01.245809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 18:14:01.245860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 18:14:01.247322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-02 18:14:01.255715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimiz